# 1. Energy data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel');
models = s.models;


In [17]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans BCM
BCM: SMSE 0.00167166, MSLL -3.2398, NLPD -0.3746
Processing energy_data kmeans PoE
PoE: SMSE 0.00204332, MSLL -2.8443, NLPD 0.0208


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [18]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 0.00138767, MSLL 2.8400, NLPD 5.7052


### 1.1.2 Grid q

In [19]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 0.00156825, MSLL 4.5966, NLPD 7.4617 - q   0.20
TERBCM: SMSE 0.00153641, MSLL 2.2046, NLPD 5.0697 - q   0.40
TERBCM: SMSE 0.00149527, MSLL 1.6313, NLPD 4.4964 - q   0.60
TERBCM: SMSE 0.00144504, MSLL 1.8732, NLPD 4.7383 - q   0.80
TERBCM: SMSE 0.00138767, MSLL 2.8400, NLPD 5.7052 - q   1.00
TERBCM: SMSE 0.00132665, MSLL 4.7455, NLPD 7.6107 - q   1.20
TERBCM: SMSE 0.00126597, MSLL 8.0933, NLPD 10.9585 - q   1.40
TERBCM: SMSE 0.00120888, MSLL 13.8293, NLPD 16.6944 - q   1.60
TERBCM: SMSE 0.00115727, MSLL 23.6710, NLPD 26.5361 - q   1.80
TERBCM: SMSE 0.00111177, MSLL 40.7442, NLPD 43.6093 - q   2.00
TERBCM: SMSE 0.00107215, MSLL 70.7946, NLPD 73.6597 - q   2.20
TERBCM: SMSE 0.00103780, MSLL 124.5082, NLPD 127.3733 - q   2.40
TERBCM: SMSE 0.00100798, MSLL 221.9833, NLPD 224.8484 - q   2.60
TERBCM: SMSE 0.00098200, MSLL 401.4138, NLPD 404.2789 - q   2.80
TERBCM: SMSE 0.00095926, MSLL 736.0671, NLPD 738.9322 - q   3.00


In [15]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = -0.2:-0.2:-3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 0.00140797, MSLL -6.6117, NLPD -3.7616 - q  -0.20
TERBCM: SMSE 0.00142063, MSLL -6.4252, NLPD -3.5751 - q  -0.40
TERBCM: SMSE 0.00143000, MSLL -6.3196, NLPD -3.4695 - q  -0.60
TERBCM: SMSE 0.00143701, MSLL -6.2507, NLPD -3.4006 - q  -0.80
TERBCM: SMSE 0.00144232, MSLL -6.2038, NLPD -3.3537 - q  -1.00
TERBCM: SMSE 0.00144641, MSLL -6.1720, NLPD -3.3219 - q  -1.20
TERBCM: SMSE 0.00144960, MSLL -6.1513, NLPD -3.3012 - q  -1.40
TERBCM: SMSE 0.00145214, MSLL -6.1392, NLPD -3.2891 - q  -1.60
TERBCM: SMSE 0.00145418, MSLL -6.1340, NLPD -3.2839 - q  -1.80
TERBCM: SMSE 0.00145583, MSLL -6.1344, NLPD -3.2844 - q  -2.00
TERBCM: SMSE 0.00145720, MSLL -6.1396, NLPD -3.2895 - q  -2.20
TERBCM: SMSE 0.00145834, MSLL -6.1488, NLPD -3.2987 - q  -2.40
TERBCM: SMSE 0.00145929, MSLL -6.1614, NLPD -3.3114 - q  -2.60
TERBCM: SMSE 0.00146010, MSLL -6.1771, NLPD -3.3270 - q  -2.80
TERBCM: SMSE 0.00146079, MSLL -6.1955, NLPD -3.3454 - q  -3.00


In [16]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = -0.2:-0.2:-3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            mu_dGP=abs(mu_dGP);
            s2_dGP=abs(s2_dGP);
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 4.91211116, MSLL 22138.7664, NLPD 22141.6165 - q  -0.20
TERBCM: SMSE 4.91123350, MSLL 15237.5511, NLPD 15240.4012 - q  -0.40
TERBCM: SMSE 4.91055297, MSLL 12333.0579, NLPD 12335.9079 - q  -0.60
TERBCM: SMSE 4.91002241, MSLL 10743.7615, NLPD 10746.6116 - q  -0.80
TERBCM: SMSE 4.90960541, MSLL 9781.7195, NLPD 9784.5696 - q  -1.00
TERBCM: SMSE 4.90927463, MSLL 9178.7718, NLPD 9181.6219 - q  -1.20
TERBCM: SMSE 4.90900972, MSLL 8806.7082, NLPD 8809.5583 - q  -1.40
TERBCM: SMSE 4.90879558, MSLL 8596.3360, NLPD 8599.1861 - q  -1.60
TERBCM: SMSE 4.90862091, MSLL 8507.3037, NLPD 8510.1538 - q  -1.80
TERBCM: SMSE 4.90847724, MSLL 8514.8896, NLPD 8517.7396 - q  -2.00
TERBCM: SMSE 4.90835813, MSLL 8603.5346, NLPD 8606.3847 - q  -2.20
TERBCM: SMSE 4.90825864, MSLL 8763.4075, NLPD 8766.2576 - q  -2.40
TERBCM: SMSE 4.90817497, MSLL 8988.4635, NLPD 8991.3136 - q  -2.60
TERBCM: SMSE 4.90810415, MSLL 9275.2942, NLPD 9278.1443 - q  -2.80
TERBCM: SMSE 4.90

### 1.1.3 Find single q

In [20]:
criterion = 'TERBCM';

In [21]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=10, init_q=1.5);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Optimizing q: remain iter - 100, q - 1.5000
grad_q_norm = -283.97
grad_q_reg =  262.28
ttb =  9141.2
Optimizing q: remain iter - 99, q - 1.4523
grad_q_norm = -598.87
grad_q_reg =  1095.5
ttb =    3.3151e+04
Optimizing q: remain iter - 98, q - 1.2409
grad_q_norm = -143.47
grad_q_reg =  190.77
ttb =    1.1839e+04
Optimizing q: remain iter - 97, q - 1.2049
grad_q_norm = -120.75
grad_q_reg =  90.130
ttb =  5302.6
Optimizing q: remain iter - 96, q - 1.1890
grad_q_norm = -142.42
grad_q_reg =  146.85
ttb =  6641.2
Optimizing q: remain iter - 95, q - 1.1619
grad_q_norm = -

Optimizing q: remain iter - 25, q - 0.7864
grad_q_norm = -10.875
grad_q_reg = -1.2244
ttb =    1.2738e+04
Optimizing q: remain iter - 24, q - 0.7868
grad_q_norm = -7.1063
grad_q_reg = -6.2525
ttb =  5767.8
Optimizing q: remain iter - 23, q - 0.7883
grad_q_norm = -21.916
grad_q_reg =  10.707
ttb =    1.2979e+04
Optimizing q: remain iter - 22, q - 0.7865
grad_q_norm =  17.616
grad_q_reg = -35.697
ttb =  9624.6
Optimizing q: remain iter - 21, q - 0.7934
grad_q_norm = -42.694
grad_q_reg =  40.218
ttb =    1.5643e+04
Optimizing q: remain iter - 20, q - 0.7861
grad_q_norm = -7.8403
grad_q_reg = -7.0722
ttb =    1.1444e+04
Optimizing q: remain iter - 19, q - 0.7877
grad_q_norm = -23.963
grad_q_reg =  16.344
ttb =    1.3697e+04
Optimizing q: remain iter - 18, q - 0.7849
grad_q_norm = -1.2763
grad_q_reg = -11.717
ttb =  1630.9
Optimizing q: remain iter - 17, q - 0.7873
grad_q_norm = -46.033
grad_q_reg =  40.652
ttb =    1.3323e+04
Optimizing q: remain iter - 16, q - 0.7799
grad_q_norm =  23.957

In [22]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.1.4 Find multi q

In [5]:
criterion = 'TERBCM';

In [6]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=10.0, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Msize =

   360
   194
   469
   503
   333
   469
   571
   424
   181
   502
   497
   338
   320
   354
   277
   174
   314
   404
   340
   617
   339
   121
   411
   253
   265
   360
   287
   323
   366
   141
   510
   505
   427
   339
   292
   345
   441
   319
   436
   638
   194
   378
   378
   500
   376
   361
   185
   163
   204
   502

Optimizing qs: remain iter - 100
qs_dist =  0.32078
Optimizing qs: remain iter - 99
qs_dist =  1.4039
Optimizing qs: remain iter - 98
qs_dist =  1.7142
Optimizing qs: remain iter - 97
qs_dist =  0.85690
Optimizing qs: remain iter - 96
qs_dist =  0.95376
Optimizing qs: remain iter - 95
qs_dist =  1.4013
Optimizing qs: remain iter - 94
qs_dist =  0.85689
Optimizing qs: remain iter - 93
qs_dist =  1.2503
Optimizing qs: remain iter - 92
qs_dist =  0.86027
Optimizing qs: remain iter - 91
qs_dist =  0.61947
Optimizing qs: remain iter - 90
qs_dist =  0.18123
Optimizing qs: remain iter - 89
qs_dist =  0.38363
Optimizing qs: rem

In [7]:
qs
save('./res/terbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.59712
   0.84884
   0.96002
   0.69641
   0.78702
   0.90751
   0.76917
   0.68362
   0.71344
   0.73931
   0.74584
   0.82664
   0.93932
   0.84403
   1.53141
   0.87370
   0.85821
   0.90990
   0.79708
   0.64736
   0.70947
   0.89274
   0.69675
   0.71837
   0.73909
   1.16778
   0.73296
   0.73784
   0.94719
   1.03798
   0.82781
   0.77732
   0.87271
   1.04805
   0.92616
   1.02643
   0.71357
   0.75949
   0.72096
   0.69311
   0.76039
   0.80850
   0.89428
   0.84194
   0.73615
   1.15269
   0.81450
   0.92329
   0.80832
   0.79026

TERBCM: SMSE 0.00123211, MSLL 0.4310, NLPD 3.2893


In [8]:
save('./res/terbcmenergydataqs_res', 'SMSE','MSLL','NLPD')

## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [9]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 0.00129128, MSLL -4.5474, NLPD -1.6891


### 1.2.2 Grid q

In [10]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGRBCM
TEGRBCM: SMSE 0.00203331, MSLL -4.5420, NLPD -1.6836 - q   0.20
TEGRBCM: SMSE 0.00182699, MSLL -4.5953, NLPD -1.7369 - q   0.40
TEGRBCM: SMSE 0.00158505, MSLL -4.6557, NLPD -1.7974 - q   0.60
TEGRBCM: SMSE 0.00139488, MSLL -4.6700, NLPD -1.8117 - q   0.80
TEGRBCM: SMSE 0.00129128, MSLL -4.5474, NLPD -1.6891 - q   1.00
TEGRBCM: SMSE 0.00127844, MSLL -4.1182, NLPD -1.2599 - q   1.20
TEGRBCM: SMSE 0.00133428, MSLL -3.0635, NLPD -0.2051 - q   1.40
TEGRBCM: SMSE 0.00142008, MSLL -0.7775, NLPD 2.0808 - q   1.60
TEGRBCM: SMSE 0.00149956, MSLL 3.9067, NLPD 6.7650 - q   1.80
TEGRBCM: SMSE 0.00155420, MSLL 13.2681, NLPD 16.1265 - q   2.00
TEGRBCM: SMSE 0.00158313, MSLL 31.8000, NLPD 34.6583 - q   2.20
TEGRBCM: SMSE 0.00159360, MSLL 68.4169, NLPD 71.2752 - q   2.40
TEGRBCM: SMSE 0.00159319, MSLL 140.8951, NLPD 143.7535 - q   2.60
TEGRBCM: SMSE 0.00158711, MSLL 284.8378, NLPD 287.6961 - q   2.80
TEGRBCM: SMSE 0.00157834, MSLL 571.8432, NLPD 574.7016 - q   3.00

### 1.2.3 Find single q

In [5]:
criterion = 'TEGRBCM';

In [10]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.01, lambda=10, init_q=1.5);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Optimizing q: remain iter - 100, q - 1.5000
grad_q_norm =  4.0899
grad_q_reg =  11.506
ttb =  116.91
Optimizing q: remain iter - 99, q - 1.4757
grad_q_norm =  7.2198
grad_q_reg =  10.509
ttb =  87.780
Optimizing q: remain iter - 98, q - 1.4528
grad_q_norm = -32.866
grad_q_reg =  61.133
ttb =  294.25
Optimizing q: remain iter - 97, q - 1.3347
grad_q_norm =  2.6342
grad_q_reg =  3.3722
ttb =  80.190
Optimizing q: remain iter - 96, q - 1.3273
grad_q_norm = -1.9507
grad_q_reg =  8.4949
ttb =  239.85
Optimizing q: remain iter - 95, q - 1.3104
grad_q_norm = -5.3381
grad_

ttb =  6.9156
Optimizing q: remain iter - 23, q - 0.7842
grad_q_norm = -5.6493
grad_q_reg =  2.4622
ttb =  45.516
Optimizing q: remain iter - 22, q - 0.7803
grad_q_norm =  0.62436
grad_q_reg =  1.4775
ttb =  33.062
Optimizing q: remain iter - 21, q - 0.7772
grad_q_norm = -2.1729
grad_q_reg =  1.8806
ttb =  36.635
Optimizing q: remain iter - 20, q - 0.7738
grad_q_norm =  0.65146
grad_q_reg =  0.86447
ttb =  25.196
Optimizing q: remain iter - 19, q - 0.7719
grad_q_norm = -0.45863
grad_q_reg =  0.62854
ttb =  24.059
Optimizing q: remain iter - 18, q - 0.7707
grad_q_norm = -0.90460
grad_q_reg =  0.20226
ttb =  11.812
Optimizing q: remain iter - 17, q - 0.7705
grad_q_norm = -0.67188
grad_q_reg =  0.16790
ttb =  10.225
Optimizing q: remain iter - 16, q - 0.7703
grad_q_norm =  2.8696
grad_q_reg =  0.13113
ttb =  9.6529
Optimizing q: remain iter - 15, q - 0.7694
grad_q_norm = -2.3887
grad_q_reg =  1.1834
ttb =  30.877
Optimizing q: remain iter - 14, q - 0.7675
grad_q_norm = -1.7969
grad_q_reg 

In [12]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [17]:
criterion = 'TEGRBCM';

In [21]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.01, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Msize =

   360
   194
   469
   503
   333
   469
   571
   424
   181
   502
   497
   338
   320
   354
   277
   174
   314
   404
   340
   617
   339
   121
   411
   253
   265
   360
   287
   323
   366
   141
   510
   505
   427
   339
   292
   345
   441
   319
   436
   638
   194
   378
   378
   500
   376
   361
   185
   163
   204
   502

Optimizing qs: remain iter - 100
qs_dist =  1.9967
Optimizing qs: remain iter - 99
qs_dist =  0.97021
Optimizing qs: remain iter - 98
qs_dist =  0.80118
Optimizing qs: remain iter - 97
qs_dist =  0.81652
Optimizing qs: remain iter - 96
qs_dist =  0.39933
Optimizing qs: remain iter - 95
qs_dist =  0.053829
Optimizing qs: remain iter - 94
qs_dist =  0.91159
Optimizing qs: remain iter - 93
qs_dist =  0.43651
Optimizing qs: remain iter - 92
qs_dist =  0.12885
Optimizing qs: remain iter - 91
qs_dist =  0.38066
Optimizing qs: remain iter - 90
qs_dist =  0.86034
Optimizing qs: remain iter - 89
qs_dist =  1.4737
Optimizing qs: 

In [22]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.29285
   1.98764
   0.73609
   0.50849
   0.88923
   0.82383
   0.68478
   0.75325
   0.78746
   0.77162
   0.57907
   0.79107
   0.81169
   0.65369
   0.62463
   0.96162
   0.58083
   0.78537
   0.76744
   0.66980
   0.75538
   0.76330
   0.59675
   0.72372
   0.76982
   0.66555
   0.79920
   0.78179
   0.69465
   0.70226
   0.67157
   0.76138
   0.37389
   0.63843
   0.82122
   0.85169
   0.61993
   0.75823
   0.67305
   0.66275
   0.78295
   0.67867
   0.40148
   0.34744
   0.40144
   1.00242
   0.74203
   0.85380
   0.66405
   0.71870

TEGRBCM: SMSE 0.00148364, MSLL -4.6735, NLPD -1.8234


In [20]:
save('./res/tegrbcmenergydataqs_res', 'SMSE','MSLL','NLPD')

## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [23]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 0.00125745, MSLL 1.5450, NLPD 4.3951


### 1.3.2 Grid q

In [24]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGPoE
TEGPoE: SMSE 0.00152096, MSLL 3.5642, NLPD 6.4143 - q   0.20
TEGPoE: SMSE 0.00146682, MSLL 1.3106, NLPD 4.1606 - q   0.40
TEGPoE: SMSE 0.00140314, MSLL 0.7040, NLPD 3.5541 - q   0.60
TEGPoE: SMSE 0.00133200, MSLL 0.8148, NLPD 3.6649 - q   0.80
TEGPoE: SMSE 0.00125745, MSLL 1.5450, NLPD 4.3951 - q   1.00
TEGPoE: SMSE 0.00118446, MSLL 3.0661, NLPD 5.9162 - q   1.20
TEGPoE: SMSE 0.00111731, MSLL 5.7987, NLPD 8.6487 - q   1.40
TEGPoE: SMSE 0.00105839, MSLL 10.5390, NLPD 13.3891 - q   1.60
TEGPoE: SMSE 0.00100816, MSLL 18.7387, NLPD 21.5888 - q   1.80
TEGPoE: SMSE 0.00096588, MSLL 33.0462, NLPD 35.8963 - q   2.00
TEGPoE: SMSE 0.00093031, MSLL 58.3434, NLPD 61.1934 - q   2.20
TEGPoE: SMSE 0.00090021, MSLL 103.7325, NLPD 106.5826 - q   2.40
TEGPoE: SMSE 0.00087454, MSLL 186.3758, NLPD 189.2259 - q   2.60
TEGPoE: SMSE 0.00085243, MSLL 338.9651, NLPD 341.8151 - q   2.80
TEGPoE: SMSE 0.00083325, MSLL 624.3551, NLPD 627.2052 - q   3.00


### 1.3.3 Find single q

In [25]:
criterion = 'TEGPoE';

In [27]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=10, init_q=1.5);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Optimizing q: remain iter - 100, q - 1.5000
grad_q_norm = -270.16
grad_q_reg =  592.48
ttb =    1.2760e+04
Optimizing q: remain iter - 99, q - 1.3846
grad_q_norm = -77.959
grad_q_reg =  76.339
ttb =  4040.2
Optimizing q: remain iter - 98, q - 1.3706
grad_q_norm = -534.41
grad_q_reg =  783.84
ttb =    2.0996e+04
Optimizing q: remain iter - 97, q - 1.2215
grad_q_norm = -108.15
grad_q_reg =  136.83
ttb =  8469.1
Optimizing q: remain iter - 96, q - 1.1958
grad_q_norm = -0.75760
grad_q_reg = -2.7291
ttb =  228.99
Optimizing q: remain iter - 95, q - 1.1964
grad_q_norm = 

grad_q_norm = -25.228
grad_q_reg =  12.269
ttb =    1.1339e+04
Optimizing q: remain iter - 24, q - 0.7790
grad_q_norm = -14.382
grad_q_reg = -14.088
ttb =    1.0916e+04
Optimizing q: remain iter - 23, q - 0.7822
grad_q_norm = -3.4964
grad_q_reg = -4.9365
ttb =  3181.6
Optimizing q: remain iter - 22, q - 0.7832
grad_q_norm = -11.674
grad_q_reg = -17.374
ttb =  5667.0
Optimizing q: remain iter - 21, q - 0.7870
grad_q_norm = -29.403
grad_q_reg = -22.708
ttb =  5550.6
Optimizing q: remain iter - 20, q - 0.7923
grad_q_norm =  11.866
grad_q_reg = -36.155
ttb =  8111.8
Optimizing q: remain iter - 19, q - 0.7994
grad_q_norm = -15.832
grad_q_reg =  20.533
ttb =    1.1097e+04
Optimizing q: remain iter - 18, q - 0.7955
grad_q_norm = -22.003
grad_q_reg = -5.3927
ttb =  4415.0
Optimizing q: remain iter - 17, q - 0.7971
grad_q_norm =  3.1854
grad_q_reg = -15.571
ttb =  6132.6
Optimizing q: remain iter - 16, q - 0.8002
grad_q_norm = -3.4649
grad_q_reg = -10.562
ttb =    1.1892e+04
Optimizing q: remai

In [29]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [30]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Msize =

   360
   194
   469
   503
   333
   469
   571
   424
   181
   502
   497
   338
   320
   354
   277
   174
   314
   404
   340
   617
   339
   121
   411
   253
   265
   360
   287
   323
   366
   141
   510
   505
   427
   339
   292
   345
   441
   319
   436
   638
   194
   378
   378
   500
   376
   361
   185
   163
   204
   502

Optimizing qs: remain iter - 100
qs_dist =  1.5041
Optimizing qs: remain iter - 99
qs_dist =  1.7275
Optimizing qs: remain iter - 98
qs_dist =  0.74272
Optimizing qs: remain iter - 97
qs_dist =  0.47266
Optimizing qs: remain iter - 96
qs_dist =  1.1795
Optimizing qs: remain iter - 95
qs_dist =  1.0376
Optimizing qs: remain iter - 94
qs_dist =  1.0511
Optimizing qs: remain iter - 93
qs_dist =  0.60297
Optimizing qs: remain iter - 92
qs_dist =  1.1140
Optimizing qs: remain iter - 91
qs_dist =  0.66522
Optimizing qs: remain iter - 90
qs_dist =  0.099498
Optimizing qs: remain iter - 89
qs_dist =  0.26207
Optimizing qs: rema

In [31]:
qs
save('./res/tegpoeenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.58045
   0.84382
   0.91310
   0.77310
   0.74332
   0.88934
   0.73395
   0.71239
   0.69959
   0.76793
   0.80011
   0.79673
   1.03848
   0.79762
   1.76134
   0.89089
   0.76726
   0.79258
   0.79486
   0.65253
   0.73700
   0.85995
   0.66722
   0.69396
   0.75737
   0.97535
   0.74619
   0.81695
   0.86614
   1.03477
   0.73302
   0.72435
   0.86696
   0.89470
   0.88851
   0.96970
   0.72331
   0.78168
   0.71919
   0.66636
   0.85685
   0.73035
   0.98696
   0.81426
   0.84020
   0.92244
   0.77273
   0.92182
   0.95294
   0.75760

TEGPoE: SMSE 0.00132153, MSLL 0.9445, NLPD 3.7946
